In [2]:
import torch
import yaml

config_path = 'allegro_model_config.yaml'

with open(config_path, "r") as stream:
    try:
        config  = yaml.load(stream)
    except yaml.YAMLError as exc:
        print('!')

/tmp/ipykernel_65240/2087197124.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config  = yaml.load(stream)


In [3]:
from nequip.model import model_from_config
from nequip.nn import RescaleOutput, GraphModuleMixin, PerSpeciesScaleShift

model = model_from_config(config, initialize=True)

In [8]:
"""Get model size in parameters"""
from torchsummary import summary

summary(model, dataset[0])

TypeError: rand(): argument 'size' must be tuple of ints, but found element of type str at pos 2

In [64]:
from nequip.model import PerSpeciesRescale

In [71]:
params = dict(
    field=AtomicDataDict.PER_ATOM_ENERGY_KEY,
    out_field=AtomicDataDict.PER_ATOM_ENERGY_KEY,
    shifts=0.,
    scales=1.,
)

params["arguments_in_dataset_units"] = False
model.func.insert_from_parameters(
    before="total_energy_sum",
    name='whatever',
    shared_params=config,
    builder=PerSpeciesScaleShift,
    params=params,
)

RuntimeError: Failed to build object with prefix `whatever` using builder `PerSpeciesScaleShift`

In [3]:
model._modules['func']

SequentialGraphNetwork(
  (one_hot): OneHotAtomEncoding()
  (radial_basis): RadialBasisEdgeEncoding(
    (basis): NormalizedBasis(
      (basis): BesselBasis()
    )
    (cutoff): PolynomialCutoff()
  )
  (spharm): SphericalHarmonicEdgeAttrs(
    (sh): SphericalHarmonics()
  )
  (allegro): Allegro_Module(
    (latents): ModuleList(
      (0): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=GraphModule)
      )
      (1): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=GraphModule)
      )
      (2): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=GraphModule)
      )
    )
    (env_embed_mlps): ModuleList(
      (0): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=GraphModule)
      )
      (1): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=GraphModule)
      )
      (2): ScalarMLPFunction(
        (_forward): RecursiveScriptModule(original_name=Graph

In [7]:
from ocpmodels.datasets.lmdb_dataset import LmdbDataset
config_ds = {'src': '/mnt/storage/ocpmodels/data/s2ef/2M/train'}

dataset = LmdbDataset(config_ds)

In [11]:
dataset[0].__dict__

{'x': None,
 'edge_index': tensor([[55, 51, 40,  ..., 23,  7, 61],
         [ 0,  0,  0,  ..., 70, 70, 70]]),
 'edge_attr': None,
 'y': -4.7176909199999955,
 'pos': tensor([[0.0000e+00, 1.5998e+00, 1.3760e+01],
         [0.0000e+00, 1.2920e+01, 9.6059e+00],
         [2.2961e+00, 4.0547e+00, 1.1683e+01],
         [2.2961e+00, 1.5375e+01, 1.1683e+01],
         [0.0000e+00, 7.2597e+00, 1.1683e+01],
         [0.0000e+00, 1.8580e+01, 7.5290e+00],
         [2.2961e+00, 9.7146e+00, 9.6059e+00],
         [2.2961e+00, 2.1035e+01, 9.6059e+00],
         [9.1776e+00, 1.5910e+00, 1.7693e+01],
         [0.0000e+00, 1.2920e+01, 1.3760e+01],
         [2.2961e+00, 4.0547e+00, 1.5837e+01],
         [2.2907e+00, 1.5284e+01, 1.5684e+01],
         [9.1835e+00, 7.3465e+00, 1.5638e+01],
         [0.0000e+00, 1.8580e+01, 1.1683e+01],
         [2.2961e+00, 9.7146e+00, 1.3760e+01],
         [2.5500e+00, 2.0997e+01, 1.3925e+01],
         [4.5922e+00, 1.5998e+00, 1.3760e+01],
         [4.5922e+00, 1.2920e+01, 9.6

In [6]:
from nequip.data import AtomicDataDict, AtomicData

In [12]:
at_num2idx = {20: 0,
                29: 1,
                47: 2,
                6: 3,
                1: 4,
                8: 5,
                11: 6,
                32: 7,
                7: 8,
                77: 9,
                75: 10,
                22: 11,
                19: 12,
                16: 13,
                50: 14,
                73: 15,
                52: 16,
                17: 17,
                82: 18,
                83: 19,
                46: 20,
                40: 21,
                13: 22,
                42: 23,
                31: 24,
                51: 25,
                30: 26,
                34: 27,
                15: 28,
                28: 29,
                79: 30,
                26: 31,
                78: 32,
                45: 33,
                39: 34,
                49: 35,
                41: 36,
                21: 37,
                14: 38,
                38: 39,
                37: 40,
                23: 41,
                72: 42,
                24: 43,
                25: 44,
                27: 45,
                55: 46,
                44: 47,
                33: 48,
                81: 49,
                80: 50,
                74: 51,
                76: 52,
                48: 53,
                43: 54,
                5: 55}

In [8]:
at_data = AtomicData.to_AtomicDataDict(dataset[0])

In [28]:
model(at_data)

{'edge_index': tensor([[55, 51, 40,  ..., 23,  7, 61],
         [ 0,  0,  0,  ..., 70, 70, 70]]),
 'pos': tensor([[0.0000e+00, 1.5998e+00, 1.3760e+01],
         [0.0000e+00, 1.2920e+01, 9.6059e+00],
         [2.2961e+00, 4.0547e+00, 1.1683e+01],
         [2.2961e+00, 1.5375e+01, 1.1683e+01],
         [0.0000e+00, 7.2597e+00, 1.1683e+01],
         [0.0000e+00, 1.8580e+01, 7.5290e+00],
         [2.2961e+00, 9.7146e+00, 9.6059e+00],
         [2.2961e+00, 2.1035e+01, 9.6059e+00],
         [9.1776e+00, 1.5910e+00, 1.7693e+01],
         [0.0000e+00, 1.2920e+01, 1.3760e+01],
         [2.2961e+00, 4.0547e+00, 1.5837e+01],
         [2.2907e+00, 1.5284e+01, 1.5684e+01],
         [9.1835e+00, 7.3465e+00, 1.5638e+01],
         [0.0000e+00, 1.8580e+01, 1.1683e+01],
         [2.2961e+00, 9.7146e+00, 1.3760e+01],
         [2.5500e+00, 2.0997e+01, 1.3925e+01],
         [4.5922e+00, 1.5998e+00, 1.3760e+01],
         [4.5922e+00, 1.2920e+01, 9.6059e+00],
         [6.8883e+00, 4.0547e+00, 1.1683e+01],
  

In [17]:
at_data

{'edge_index': tensor([[55, 51, 40,  ..., 23,  7, 61],
         [ 0,  0,  0,  ..., 70, 70, 70]]),
 'pos': tensor([[0.0000e+00, 1.5998e+00, 1.3760e+01],
         [0.0000e+00, 1.2920e+01, 9.6059e+00],
         [2.2961e+00, 4.0547e+00, 1.1683e+01],
         [2.2961e+00, 1.5375e+01, 1.1683e+01],
         [0.0000e+00, 7.2597e+00, 1.1683e+01],
         [0.0000e+00, 1.8580e+01, 7.5290e+00],
         [2.2961e+00, 9.7146e+00, 9.6059e+00],
         [2.2961e+00, 2.1035e+01, 9.6059e+00],
         [9.1776e+00, 1.5910e+00, 1.7693e+01],
         [0.0000e+00, 1.2920e+01, 1.3760e+01],
         [2.2961e+00, 4.0547e+00, 1.5837e+01],
         [2.2907e+00, 1.5284e+01, 1.5684e+01],
         [9.1835e+00, 7.3465e+00, 1.5638e+01],
         [0.0000e+00, 1.8580e+01, 1.1683e+01],
         [2.2961e+00, 9.7146e+00, 1.3760e+01],
         [2.5500e+00, 2.0997e+01, 1.3925e+01],
         [4.5922e+00, 1.5998e+00, 1.3760e+01],
         [4.5922e+00, 1.2920e+01, 9.6059e+00],
         [6.8883e+00, 4.0547e+00, 1.1683e+01],
  

In [25]:
at_data['atom_types'] = torch.LongTensor([at_num2idx[number.item()] for number in at_data['atomic_numbers']])
at_data['edge_cell_shift'] = at_data.pop('cell_offsets')

In [27]:
at_data['edge_cell_shift'] = at_data['edge_cell_shift'].float()

In [24]:
min_Z = 1
max_Z = 83

Z_to_index = torch.full(
    size=(1 + max_Z - min_Z,), fill_value=-1, dtype=torch.long
)

In [25]:
at_data['atom_types'] = Z_to_index[at_data['atomic_numbers']]

IndexError: tensors used as indices must be long, byte or bool tensors